In [ ]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from google import genai
import json
from pydantic import BaseModel
import time
import pymupdf
import tempfile


In [61]:
anexos = pd.read_excel('docs/jusbrasil.xlsx', sheet_name="Anexos")

In [62]:
anexos_copia = anexos.copy().loc[:,["processoID", "processoAnexoID", "Download copia", "Tipo de anexo"]]
sentencas = anexos_copia.loc[anexos["Tipo de anexo"].isin(["SENTENCA"]),
                                           ["processoID", "processoAnexoID", "Download copia"]]
sentencas.head()

,processoID,processoAnexoID,Download copia
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...


In [63]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto)
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = sentencas[sentencas['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)
acessos_negados.head()

,processoID,processoAnexoID,Download copia
1293,471515190,727395468,http://jud-anexos.digesto.com.br/3a9fbbe2c4f69...
1302,471515190,727395467,http://jud-anexos.digesto.com.br/7767cbbc02e35...
2876,595998611,1403409515,http://jud-anexos.digesto.com.br/0ec3e0c015d72...
2877,595998611,1403409516,http://jud-anexos.digesto.com.br/385306277d658...
8000,690025576,2015456650,http://jud-anexos.digesto.com.br/fb01c9fd839aa...


In [64]:
# Dropando os acessos negados
sentencas_acessaveis = sentencas[sentencas["processoAnexoID"].isin(acessos_negados["processoAnexoID"]) == False]

In [65]:
# Verificando quantidade de processos após o drop
sentencas_acessaveis["processoID"].unique().shape

(223,)

In [70]:
sentencas_acessaveis.to_excel("docs/sentencas_acessaveis.xlsx", index=False)

In [66]:
ids_processos = sentencas_acessaveis["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

Número de processos com mais de uma sentença: 84


In [67]:
def verifica_dano_ambiental(texto):
    """
    Funcao que verifica se o texto possui dano ambiental.
    Se houver, justifica a resposta. Caso contrário, responde "não há dano ambiental".
    """
    
    prompt = f"""
    Você é um especialista em direito ambiental. 
    Você deve analisar o seguinte texto e verificar se há dano ambiental. 
    Se houver, justifique sua resposta. Caso contrário, responda "não há dano ambiental".

    Texto: {texto}
    """
    
    class FormatoResposta(BaseModel):
        isDanoAmbiental: bool
        justificativa: str

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': FormatoResposta
            # 'max_output_tokens': 500,
            # 'temperature': 1.0
        }
    )
    time.sleep(2)
    return resposta

In [68]:
respostas = []

for id in ids_processos:
    linhas_correspondentes = sentencas.loc[sentencas['processoID'] == id]
    texto = ''

    for index, row in linhas_correspondentes.iterrows():
        processoAnexoID = row['processoAnexoID']
        link = row['Download copia']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    
        resposta_prompt = verifica_dano_ambiental(texto)
        resposta_prompt = json.loads(resposta_prompt.text)
        resposta_prompt["processoAnexoID"] = processoAnexoID
        resposta_prompt["link_referencia"] = link
        resposta_prompt["processoID"] = id
        respostas.append(resposta_prompt)

respostas

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}